In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Korea", country_b="Mexico")

chat.predict(prompt)

'The distance between Korea and Mexico is approximately 11,000 kilometers (6,835 miles) when measured in a straight line.'

In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕, 나는 {name}이야."),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="세종",
    country_a="Chosun",
    country_b="Rome",
)

chat.predict_messages(prompt)

AIMessage(content='제 이름은 세종이에요. 조선과 로마 사이의 거리는 약 8,000km 정도 되요.')